### Compiling with the mqtt_as library as frozen bytecode

Instructions from:
    https://www.esp8266.com/wiki/doku.php?id=toolchain

Make the esp C-compiler
> conda deactivate
> git clone --recursive https://github.com/pfalcon/esp-open-sdk
> cd esp-open-sdk
> make

Copy across the modules
> cp /home/julian/extrepositories/micropython-mqtt/mqtt_as/mqtt_as.py /home/julian/extrepositories/micropython/ports/esp8266/modules/

> cp /home/julian/extrepositories/micropython-lib/uasyncio/uasyncio/ /home/julian/extrepositories/micropython/ports/esp8266/modules/

> cp /home/julian/extrepositories/micropython-lib/uasyncio.core/uasyncio/core.py /home/julian/extrepositories/micropython/ports/esp8266/modules/uasyncio/

> cp /home/julian/extrepositories/micropython-lib/uasyncio.queues/uasyncio/queues.py /home/julian/extrepositories/micropython/ports/esp8266/modules/uasyncio/

> cp /home/julian/extrepositories/micropython-lib/umqtt.simple/umqtt/simple.py /home/julian/extrepositories/micropython/ports/esp8266/modules/umqtt/

> cp /home/julian/extrepositories/micropython-lib/umqtt.robust/umqtt/robust.py /home/julian/extrepositories/micropython/ports/esp8266/modules/umqtt/

> export PATH="/home/julian/extrepositories/esp-open-sdk/xtensa-lx106-elf/bin/:$PATH"

Make the file
> cd /home/julian/extrepositories/micropython/ports/esp8266

> make -j4


/home/julian/extrepositories/micropython/ports/esp8266/build-GENERIC/firmware-combined.bin


In [20]:
%esptool erase

Executing:
  esptool.py --port /dev/ttyUSB1 erase_flash

esptool.py v2.0
Connecting....
[Press the PRG button now if required]
Detecting chip type... ESP8266
Chip is ESP8266
Uploading stub...
Running stub...
Stub running...
Erasing flash (this may take a while)...
Chip erase completed successfully in 11.2s
Hard resetting...


In [21]:
%esptool esp8266 /home/julian/extrepositories/micropython/ports/esp8266/build-GENERIC/firmware-combined.bin

Executing:
  esptool.py --port /dev/ttyUSB1 --baud 460800 write_flash --flash_size=detect -fm dio 0 /home/julian/extrepositories/micropython/ports/esp8266/build-GENERIC/firmware-combined.bin

esptool.py v2.0
Connecting....
[Press the PRG button now if required]
Detecting chip type... ESP8266
Chip is ESP8266
Uploading stub...
Running stub...
Stub running...
Changing baud rate to 460800
Changed.
Configuring flash size...
Auto-detected Flash size: 4MB
Flash params set to 0x0240
Compressed 618180 bytes to 403189...
Wrote 618180 bytes (403189 compressed) at 0x00000000 in 9.3 seconds (effective 531.9 kbit/s)...
Hash of data verified.

Leaving...
Hard resetting...


In [22]:
%serialconnect
help("modules")

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.
__main__          itertools         uasyncio/queues   upip_utarfile
_boot             lwip              uasyncio/synchro  urandom
_onewire          machine           ubinascii         ure
_webrepl          math              ucollections      uselect
apa102            micropython       ucryptolib        usocket
btree             mqtt_as           uctypes           ussl
builtins          neopixel          uerrno            ustruct
collections/deque network           uhashlib          utime
dht               ntptime           uheapq            utimeq
ds18x20           onewire           uio               uwebsocket
esp               port_diag         ujson             uzlib
flashbdev         sys               umqtt/robust      webrepl
framebuf          uarray            umqtt/simple      webrepl_setup
gc                uasyncio/__init__ uos               websocket_helper
inisetup          uasyncio/core     upip
Plus any modules on the

In [4]:
# Small ESP8266

#      RST          |  GPIO1   TX
#       A0          |  GPIO3   RX
#       D0  GPIO16  |  GPIO5   D1  SCL
# SCK   D5  GPIO14  |  GPIO4   D2  SDA
# MISO  D6  GPIO12  |  GPIO0   D3
# MOSI  D7  GPIO13  |  GPIO2   D4  LED
# SS    D8  GPIO15  |          GND
#      3V3          |          5V

# (not able to get the mDNS to work for esp8266)

No serial connected
  %serialconnect to connect
  %esptool to flash the device
  %lsmagic to list commands

In [19]:
# Setting up the webrepl hotspot with password=wpass
%serialconnect --raw
%readbytes 

# Print out the hotspot name (while we are here)
%writebytes "import network, ubinascii\r\n"
%readbytes
%writebytes "\r\n"
%readbytes
%writebytes "'MicroPython-%s'%ubinascii.hexlify(network.WLAN().config('mac')[-3:]).decode()\r\n"
%comment "This is the ESP8266's hotspot name"
%readbytes
%writebytes "\r\n"
%readbytes

# soft reset so import works
%writebytes "\x04"
%readbytes
%writebytes "import webrepl_setup\r\n"
%readbytes 
%writebytes "E\r\n"
%readbytes

# uncomment if second time (with additional question of changing password)
#%writebytes "y\r\n"
#%readbytes

# set the password
%writebytes "wpass\r\n"
%readbytes
%writebytes "wpass\r\n"
%readbytes

#%writebytes "y\r\n"
#%readbytes

%writebytes "\x04"
%readbytes
%disconnect --raw

Connecting to --port=/dev/ttyUSB6 --baud=115200 
import network, ubinascii
>>> 
>>> This is the ESP8266's hotspot name'MicroPython-%s'%ubinascii.hexlify(network.WLAN().config('mac')[-3:]).decode()
'MicroPython-2d0c59'
>>> 
>>> 
MPY: soft reboot
Performing initial setup
MicroPython v1.11-543-g48b25a841 on 2019-10-31; ESP module with ESP8266
Type "help()" for more information.
>>> import webrepl_setup
WebREPL daemon auto-start status: disabled

Would you like to (E)nable or (D)isable it running on boot?
(Empty line to quit)
> E
To enable WebREPL, you must set password for it
New password (4-9 chars): wpass
Confirm password: wpass

Closing serial Serial<id=0x7fc8da101470, open=True>(port='/dev/ttyUSB6', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
